# Image to poem

## Description
Takes an image as input, identifies it using the Google Vision API, and creates a poem about it  
  
Uses the [Google Vision API](https://cloud.google.com/vision/) to **detect the entity** in the image  
Uses the [datamuse API](http://www.datamuse.com/api/) for the **word logic**  
**Pictures** freely available on [Unsplash](https://unsplash.com/)

## Todo :
- Add poetry styles
- Syllable counts
- Paste text on image & export
- Add custom variables (name for person/object/pet...)

### Import libraries

In [1]:
import io
import os

import requests
import html

from google.cloud import vision
from google.cloud.vision import types

Set environment variable to the API key

### Google Vision

Creating a **Google Vision instance** using an **API key**

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = './ressources/params.json'

# Instantiates a client
client = vision.ImageAnnotatorClient()

Loads the **image file**

We're using the following image of a bike :
![Image from Unsplash.com](./images/bike.jpg "Image from Unsplash.com")

In [42]:
# The name of the image file to annotate
file_name = './images/bike.jpg'

# Loads the image into memory
with io.open(file_name, 'rb') as image_file:
    content = image_file.read()

image = types.Image(content=content)

Prints out the **detected entities** and their **score**

In [43]:
# Performs label detection on the image file
response = client.label_detection(image=image)
labels = response.label_annotations

print('Labels : score')
for label in labels:
    print(f'{label.description} : {round(label.score * 100, 1)}%')

Labels : score
bicycle : 98.9%
land vehicle : 98.6%
road bicycle : 97.8%
yellow : 92.5%
bicycle wheel : 91.8%
wall : 89.6%
bicycle frame : 87.4%
vehicle : 83.4%
snapshot : 81.8%
sports equipment : 81.6%


### Rhymes and related words using DataMuse

Displays a few functionalities of the **datamuse API**

In [48]:
word = labels[0].description

with requests.get('http://api.datamuse.com/words?' + 'rel_jjb=' + word) as f:
    adj_json = f.json()
    adj = [result['word'] for result in adj_json]

with requests.get('http://api.datamuse.com/words?' + 'ml=' + word) as f:
    ml_json = f.json()
    ml = [result['word'] for result in ml_json]

with requests.get('http://api.datamuse.com/words?' + 'rel_trg=' + word) as f:
    trg_json = f.json()
    trg = [result['word'] for result in trg_json]

with requests.get('http://api.datamuse.com/words?' + 'rel_rhy=' + word) as f:
    rhy_json = f.json()
    rhy = [result['word'] for result in rhy_json]

    print('Word : ' + word)
    print('')
    for i in range(len(adj[:5])) : print('Commonly used adjectives : ' + adj[i] + ' ' + word)

    for i in range(len(trg[:5])) : 
        if i != 0 : 
            print('Common words following ' + word + ' : ' +trg[i])

    for i in range(len(ml[:5])) : 
        if i != 0 : 
            print('Similar words : ' + ml[i])

    for i in range(len(rhy[:5])) : 
        if i != 0 : 
            print('Rhymes with ' + word + ' : '+ rhy[i])


Word : bicycle

Commonly used adjectives : new bicycle
Commonly used adjectives : stationary bicycle
Commonly used adjectives : old bicycle
Commonly used adjectives : first bicycle
Commonly used adjectives : motor bicycle
Common words following bicycle : racer
Common words following bicycle : handlebars
Common words following bicycle : aba
Common words following bicycle : bike
Similar words : pedal
Similar words : wheel
Similar words : cycle
Similar words : bikes
Rhymes with bicycle : thy sickle


### Making a test poem

In [49]:
for i in [0]:
    word = labels[i].description
    try : 
        word = word.split(' ')[-1]
    except:
        pass
    with requests.get('http://api.datamuse.com/words?' + 'rel_jjb=' + word) as f:
        adj_json = f.json()
        adj = [result['word'] for result in adj_json]
        
    with requests.get('http://api.datamuse.com/words?' + 'ml=' + word) as f:
        ml_json = f.json()
        ml = [result['word'] for result in ml_json]
        
    with requests.get('http://api.datamuse.com/words?' + 'rel_trg=' + word) as f:
        trg_json = f.json()
        trg = [result['word'] for result in trg_json]

    with requests.get('http://api.datamuse.com/words?' + 'rel_rhy=' + word) as f:
        rhy_json = f.json()
        rhy = [result['word'] for result in rhy_json]
        
        print('Title : Tale of the ' + word)
        print('')
        print('Is this the ' + adj[0] + ' ' + word)
        print('Very ' + adj[1] + ' ' + rhy[0])        
        print('The ' + adj[2] + ' ' + word)
        print('Very ' + adj[3] + ' ' + rhy[1])
        print("Not unlike a " + adj[4] + ' ' + ml[0] + '; ' + trg[0])
        

Title : Tale of the bicycle

Is this the new bicycle
Very stationary icicle
The old bicycle
Very first thy sickle
Not unlike a motor bike; motocross
